## 卷积神经网络 & TensorBoard

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
import urllib

## 获取数据

In [3]:
mnist = input_data.read_data_sets('MNIST_data/',one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## 定义模型

In [ ]:
def weight_initial(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

def bias_initial(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1],strides=[1,2,2,1], padding='SAME')

# 输入层
x = tf.placeholder(tf.float32, name='input_x')
y = tf.placeholder(tf.float32, name='input_y')
x_image = tf.reshape(x, [-1,28,28,1])

# 卷积层1
W_conv1 = weight_initial([5,5,1,32])
b_conv1 = bias_initial([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1)+b_conv1)
h_pool1 = max_pool_2x2(h_conv1) # 14x14

# 卷积层2
W_conv2 = weight_initial([5,5,32,64])
b_conv2 = bias_initial([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2)+b_conv2)
h_pool2 = max_pool_2x2(h_conv2)  # 7x7

# 全连接层
W_fc1 = weight_initial([7*7*64,512])
b_fc1 = bias_initial([512])
h_pool2_flat = tf.reshape(h_pool2, [-1,7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Readout layer
W_fc2 = weight_initial([512,10])
b_fc2 = bias_initial([10])
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [ ]:
# loss function 
with tf.name_scope('loss'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=y_conv))
    tf.summary.scalar('loss', cross_entropy)
    
with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(1e-3)
    train_step = optimizer.minimize(cross_entropy)

with tf.name_scope('accuracy'):
    corrent_prediction = tf.equal(tf.arg_max(y_conv,1),tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(corrent_prediction,tf.float32))
    tf.summary.scalar('accuracy', accuracy)

In [ ]:
def make_hparam_string(learning_rate, use_two_fc, use_two_conv):
    conv_param = "conv=2" if use_two_conv else "conv=1"
    fc_param = "fc=2" if use_two_fc else "fc=1"
    return "lr_%.0E,%s,%s" % (learning_rate, conv_param, fc_param)

LOG_DIR = './tensorboard/board003/'
hparam = make_hparam_string(1E-4, True, True)
print('hparam:',hparam)


In [ ]:
batch_size = 50
n_batch = mnist.train.num_examples // batch_size

summ_merge = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(LOG_DIR+hparam,sess.graph)
    for epoch in range(3):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            if (batch+1)%20 == 0:
                train_accuracy = accuracy.eval(feed_dict={x: batch_xs, y:batch_ys, keep_prob:1.0})
                s = sess.run(summ_merge, feed_dict={x:batch_xs, y:batch_ys,keep_prob:0.5})
                writer.add_summary(s, epoch*n_batch+batch)
                print('step %d-%d, training accuracy %g' % (epoch,batch, train_accuracy))
            train_step.run(feed_dict={x: batch_xs, y:batch_ys, keep_prob:0.5})
            
    print('test accuracy %g' % accuracy.eval(feed_dict={
      x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0}))

In [ ]:
n_batch